In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../input"))

In [ ]:
import torch 
import numpy as np

#check if CUDA is available
train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  training on CPU....')
else:
    print('CUDA is available.  training on GPU..')

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

#number of subprocess to use for data loading
num_workers = 0
#how many samples per batch to load
batch_size = 20
#percentage of validation to be used as validation
valid_size = 0.2

#convert data to a normalized
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
#choose the training and test datasets
train_data = datasets.CIFAR10('data', train = True, download = True, transform = transform)
test_data =  datasets.CIFAR10('data', train = False, download = True, transform = transform)


#obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train)) #here range will accept the integer from 0 to len of num_train and arranged them as list
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train)) #here floor will remove decimal values
print(split)
train_idx, valid_idx = indices[split:], indices[:split]
print(len(train_idx))

#define the smaplers for obtaining the traning and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

#choose the training and test dataloaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
                                           sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, 
                                           num_workers = num_workers)

#specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

# 1. visualize a Batch of training data

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
#helper function to un-nrmalise and display an image
def imshow(img):
    img = img / 2 + 0.5 #unnormalize  
    plt.imshow(np.transpose(img,(1,2,0)))

In [ ]:
#obtain one batch of training
dataiter = iter(train_loader)
images, labels = dataiter.next()
images = images.numpy() # convert images to numpy for display


#plot the images in the batch, along the coressponding labels
fig = plt.figure(figsize=(25,4))
#display 20 images
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    
    imshow(images[idx])
    ax.set_title(classes[labels[idx]])

# 1. 1  view an image in more details

In [ ]:
rgb_img = np.squeeze(images[3])
channels =['red channel', 'green channel', 'blue channel']

fig = plt.figure(figsize =(36,36))
for idx in np.arange(rgb_img.shape[0]):
    ax = fig.add_subplot(1, 3, idx+1)
    img = rgb_img[idx]
    ax.imshow(img, cmap ='gray')
    ax.set_title(channels[idx])
    width, height = img.shape
    thresh = img.max()/2.5
    for x in range(width):
        for y in range(height):
            val = round(img[x][y],2) if img[x][y] != 0 else 0
            ax.annotate(str(val), xy =(y,x),
            horizontalalignment ='center',
            verticalalignment ='center', size =8,
            color='white' if img[x][y]< thresh else 'black')
                                    

# 1.2 CNN layers architecture

In [ ]:

import torch.nn.functional as F

#define the CNN layers
class Net(nn.Module): #(nn.linearit does the linear transformation( inputs(x) * weights + bias)
    def __init__(self):
        #init define all the layers in CNN
        super(Net, self).__init__()  # basically  here super.init trys to call base class having init function ('we can pass argument inside it')
        #convolutional layer (sees 32x32x3 image tensor)
        self.conv1 = nn.Conv2d(3, 16, 3, padding =1)
        #convontoinal layer (sees 16x16x16 tensor)
        self.conv2 = nn.Conv2d(16, 32, 3, padding =1)
        #convontoinal layer (sees 8x8x32 tensor)
        self.conv3 = nn.Conv2d(32, 64, 3, padding =1)
        
        
        #max pooling layer
        self.pool = nn.MaxPool2d(2,2)
#here 64 will be output of conv last layer as a depth
        #linear layer(64 * 4 * 4 -> 500)
        self.fc1 = nn.Linear(64 * 4 * 4, 500)
        #linear layer (500 -> 10)
        self.fc2 = nn.Linear(500, 10)
        #dropout layer 
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        #adding the activation function
        print(x.shape)
        #as we are appling pooling lyer aftereach conv layer,so it will be 32X32 -> 16X16 ->8x8 =4*4
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        # ABOVE 4*4 WILL BE TAKEN AS SIZE ND 64 AS INPUT DEPTH
        #flatten the image input
        x = x.view(-1, 64*4*4)
        #add dropout layer # PREVENTING overfitting
        x = self.dropout(x)
        
        
        #add 1st hidden alyer of fuuly connected
        x = F.relu(self.fc1(x))
        #add dropout layer
        x = self.dropout(x)
        #add 2nd hidden layer
        x = self.fc2(x)
        return x
        
#create a complete CNN
model = Net()
print(model)

#move tensor to GPU if CUDA is available
if train_on_gpu:
    model.cuda()
        
        

In [ ]:
import torch.optim as optim

#specify the loss function
criterion = nn.CrossEntropyLoss()

#specify optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.01 )

# 1. 4 TRAin of network

In [ ]:

n_epochs = 2

valid_loss_min = np.Inf # track change in validation loss

for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
   ###################
    # train the model #
    ###################
    model.train()
    for data, target in train_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)
    
    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    
    #calculte the average loss
    train_loss = train_loss/len(train_loader.sampler)
    valid_loss = valid_loss/len(valid_loader.sampler)
    
    #print training /validation statics
    print('epoch: {} \tTraining Loss: {:6f} \tValidation Loss: {:6f}' .format(epoch, train_loss, valid_loss))

    #save the model if validaton loss has decreased
    if valid_loss <= valid_loss_min:
     print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
    torch.save(model.state_dict(), 'model_cifar.pt')
    valid_loss_min = valid_loss
   

# 1. 4  TEST the trained model

In [ ]:

test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
#iterate over test data
for data, target in test_loader:
    #move tensor to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
        #forward pass
        output = model(data)
        #calculate the loss
        loss = criterion(output, target)
        #update test loss
        test_loss += loss.item()*data.size(0)
        #convert output probablitites to predicted class
        _, pred = torch.max(output, 1)    
        #compare prediction to true label
        correct_tensor = pred.eq(target.data.view_as(pred))
        correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())     
    #calculate test accuracy
        for i in range(batch_size):
            label = target.data[i]
            class_correct[label] += correct[i].item()
            class_total[label]  +=1
            
    #average test loss
    test_loss = test_loss/len(test_loader.dataset)
    print('test loss:'.format(test_loss))
    
    for i in range(10):
        if class_total[i] > 0:
            print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
        else:
             print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))
            